In [2]:
import pandas as pd  
import requests  
import re  
from bs4 import BeautifulSoup  
import json
import time 

In [ ]:
url_1="https://sousuo.www.gov.cn/search-gov/data?t=zhengcelibrary&q="
url_3="&timetype=&mintime=&maxtime=&sort=score&sortType=1&searchfield=title&pcodeJiguan=&childtype=&subchildtype=&tsbq=&pubtimeyear=&puborg=&pcodeYear=&pcodeNum=&filetype=&p="
url_2="&n=5&inpro=&bmfl=&dup=&orpro=&type=gwyzcwjk"
header={
'Host': 'sousuo.www.gov.cn',
'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:128.0) Gecko/20100101 Firefox/128.0',
'Accept': 'application/json, text/plain, */*',
'Accept-Language': 'zh-CN,zh;q=0.8,zh-TW;q=0.7,zh-HK;q=0.5,en-US;q=0.3,en;q=0.2',
'Accept-Encoding': 'gzip',
'Connection': 'keep-alive',
'Referer': 'https://sousuo.www.gov.cn/zcwjk/policyDocumentLibrary?q=%E4%B9%A1%E6%9D%91%E6%8C%AF%E5%85%B4&t=zhengcelibrary&orpro=',
'Cookie': 'arialoadData=false',
'Sec-Fetch-Dest': 'empty',
'Sec-Fetch-Mode': 'cors',
'Sec-Fetch-Site': 'same-origin',
'Priority': 'u=1'
}
search_list=['乡村振兴','农村发展','乡村发展']
data=[]
for search in search_list:
    for i in range(25):
        try:
            r=requests.get(url_1+search+url_3+str(i+1)+url_2,headers=header)
            r=r.json()
            keys=[]
            for i in r['searchVO']['catMap']:
                keys.append(i)
            result=[]
            for key in keys:
                if key!='otherfile':
                    result+=r['searchVO']['catMap'][key]['listVO']
                    data+=result
            print('\r Get '+str(len(result))+' | Total -> '+str(len(data)))
        except Exception as e:
            print('ERROR-> page'+str(i+1))
            print(e)
            continue
        time.sleep(1)

In [35]:
df1=pd.read_excel('./xczx.xlsx')
df2=pd.read_excel('./xczx_2.xlsx')
df3=pd.read_excel('./xczx_3.xlsx')

In [36]:
df=pd.concat([df1[df1.columns[1:]],df2[df2.columns[1:]],df3[df3.columns[1:]]],axis=0)

In [37]:
df_unique = df.drop_duplicates(subset=['id'], keep='first')
len(df_unique)

288

In [38]:
def clean_text(text:str):
    clean_text = text.replace('\u3000','')
    clean_text = re.sub(r'<.*?>', '', clean_text)
    return clean_text
def df_to_dict(df:pd.DataFrame):
    columns=df.columns[1:]
    r=[{} for i in range(len(df))]
    for i in range(len(df)):
        for u in columns:
            r[i][u]=df[u][i]
    return r

In [39]:
df_unique[:2]

,piclinksurl,code,pcode,source,title,totalCount,pubtimeStr,fwzh,imgurl,dateType,...,zhengtype,personal,gname,subjectword,url,childtype,pubtime,shixiao,puborg,syqt
0,NaN,NaN,国办函〔2021〕51号,NaN,国务院办公厅转发国家<em>乡村</em><em>振兴</em>局中央农办财政部关于加强扶贫...,0,2021.06.04,NaN,NaN,True,...,NaN,False,NaN,NaN,http://www.gov.cn/zhengce/zhengceku/2021-06/04...,民政、扶贫、救灾\扶贫,1622789760000,NaN,国务院办公厅,NaN
1,NaN,NaN,国发〔2019〕12号,NaN,国务院关于促进<em>乡村</em>产业<em>振兴</em>的指导意见,0,2019.06.28,NaN,NaN,True,...,NaN,False,NaN,NaN,http://www.gov.cn/zhengce/zhengceku/2019-06/28...,农业、林业、水利\农业、畜牧业、渔业,1561712100000,NaN,国务院,NaN


In [40]:
pd.DataFrame(df_unique[['title','pcode','puborg','pubtimeStr']]).to_excel('final_simply.xlsx')

In [51]:
dcit=df_to_dict(pd.read_excel('./final_simply.xlsx'))

In [52]:
for ind in range(len(dcit)):
    dcit[ind]['序号']=str(ind+1)
    dcit[ind]['标题']=clean_text(dcit[ind]['title'])
    dcit[ind]['时间']=dcit[ind]['pubtimeStr']
    dcit[ind]['发文字号']=dcit[ind]['pcode']
    try:
        dcit[ind]['puborg']+=''
    except Exception as e:
        dcit[ind]['puborg']='新华社'
    dcit[ind]['来源']=dcit[ind]['puborg']
    del dcit[ind]['title']
    del dcit[ind]['pubtimeStr']
    del dcit[ind]['pcode']
    del dcit[ind]['puborg']

In [53]:
pd.DataFrame(dcit).to_excel('./show.xlsx')